In [18]:
import pandas as pd
df = pd.read_csv('./input/spins_and_freespins.csv', names=["id", "event_time", "play_bet", "play_win"])
df.sort_values(by=['id', 'event_time'])
player_ids = df['id'].unique()
df.head()

,id,event_time,play_bet,play_win
0,28,2020-02-28 03:02:48,2250000,0.0
1,28,2020-02-28 03:02:51,2250000,0.0
2,28,2020-02-28 03:02:54,2250000,3375000.0
3,28,2020-02-28 03:03:03,2250000,1125000.0
4,28,2020-02-28 03:03:09,2250000,0.0


In [19]:
df['value'] = (df[['id', 'play_bet']] != df[['id', 'play_bet']].shift()).any(axis=1)
df['value'] = df['value'].cumsum()
df = df[df['play_bet']==0]
df = df.groupby(['id','value'])['play_win'].sum().reset_index()
df.head()

,id,value,play_win
0,28,2,0.0
1,28,4,9000000.0
2,28,8,0.0
3,28,10,4500000.0
4,28,12,14625000.0


In [20]:
df = df.groupby('id')['play_win'].max().reset_index()
df.head()

,id,play_win
0,28,45632250.0
1,308,937500.0
2,1356,865987500.0
3,1616,135217500.0
4,1784,63497250.0


In [21]:
#PlayerId,ChipsAwarded,DollarsAwarded
df['PlayerId'] = df['id']
df['ChipsAwarded'] = df['play_win'] * 3
df['DollarsAwarded'] = df['ChipsAwarded'] / 10000000 
df = df[["PlayerId","ChipsAwarded","DollarsAwarded"]]

In [23]:
remaining_list  = [[id,0,0] for id in player_ids if id not in list(df['PlayerId'])]
remaining_df = pd.DataFrame(remaining_list, columns =['PlayerId', 'ChipsAwarded','DollarsAwarded'])
res = df.append(remaining_df, ignore_index=True)

/var/folders/lk/dz898tfx1zz34s7stmtpdj1r0000gn/T/ipykernel_76047/4038934990.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = df.append(remaining_df, ignore_index=True)


In [24]:
res.shape

(3047, 3)

In [47]:
df.to_csv('./output/player_award_pandas.csv', index=False)